# 02 Regression with PyTorch 
## Introduction
In this exercise, we fit a neural network model to 1D data.

We shall use the machine learning model 

$$f(\mathbf{x}, \theta) = \,\mathbf{b}_1 + \mathbf{w}_1 \, \mbox{relu}(\mathbf{b}_0 + \mathbf{w}_0 \, \mathbf{x}),$$

where $\mathbf{b}$ and $\mathbf{w}$ (the biases and weights) are the parameters $\theta$ of the model and $\mbox{relu}(x)$ is a function applied to every element $x_i$ of its tensor argument (i.e., applied element-wise) defined by

\begin{align*}
\mbox{relu}(x) & = \begin{cases}
    x, & \text{if } x \gt 0\\
    0              & \text{otherwise}.
\end{cases}
\end{align*}


In [1]:
import os, sys

# the standard module for tabular data
import pandas as pd

# the standard module for array manipulation
import numpy as np

# the standard modules for high-quality plots
import matplotlib as mp
import matplotlib.pyplot as plt

#  a function to save results
import joblib as jb

# pytorch
import torch

#  split data into a training set and a test set
from sklearn.model_selection import train_test_split

# to reload modules
import importlib

%matplotlib inline

In [2]:
# update fonts
FONTSIZE = 14
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : FONTSIZE}
mp.rc('font', **font)

# set a seed to ensure reproducibility
seed = 128
rnd  = np.random.RandomState(seed)

### Load data

In [5]:
data = jb.load('data_02.db')
data[:10]

,target,x
0,2.779505,0.819436
1,2.779505,-0.558397
2,2.779505,-0.660444
3,2.779505,-0.033374
4,2.779505,0.776216
5,2.779505,-0.875471
6,2.779505,-0.881089
7,2.779505,0.376558
8,2.779505,0.354957
9,2.779505,0.159820


### Train, validation, and test sets
There is some confusion in terminology regarding validation and test samples (or sets). We shall adhere to the defintions given here https://machinelearningmastery.com/difference-test-validation-datasets/):
   
  * __Training Dataset__: The sample of data used to fit the model.
  * __Validation Dataset__: The sample of data used to decide 1) whether the fit is reasonable (e.g., the model has not been overfitted), 2) decide which of several models is the best and 3) tune model hyperparameters.
  * __Test Dataset__: The sample of data used to provide an unbiased evaluation of a final model fit on the training dataset.

The validation set will be some small fraction of the training set and will be used to decide when to stop the training.

In [6]:
# Fraction of the data assigned as test data
fraction = 0.20
# Split data into a part for training and a part for testing
train_data, test_data = train_test_split(data, test_size=fraction)

# Split the training data into a part for training (fitting) and
# a part for validating the training.
fraction = 0.125
train_data, valid_data = train_test_split(train_data, test_size=fraction)

print('train set size:        %6d' % train_data.shape[0])
print('validation set size:   %6d' % valid_data.shape[0])
print('test set size:         %6d' % test_data.shape[0])

train set size:         35000
validation set size:     5000
test set size:          10000


Split data into targets $t$ and inputs $\mathbf{x}$

In [8]:
def split_t_x(df):
    # change from pandas dataframe format to a numpy 
    # array of the specified types
    t = np.array(df['target'])
    x = np.array(df['x'])
    return (t, x)

train_t, train_x = split_t_x(train_data)
valid_t, valid_x = split_t_x(valid_data)
test_t,  test_x  = split_t_x(test_data)

print(train_x[:3], end='\n\n') 
print(valid_x[:3], end='\n\n')
print(test_x[:3],  end='\n\n')

[ 0.95300978 -0.60370799  0.06827101]

[-0.59446741  0.49942012  0.78096059]

[0.73273797 0.8346517  0.99478322]



### Some useful tensor operations

In [43]:
# create a float tensor
a = torch.Tensor([[1,2], [3,4]])
print(a)

# create an integer tensor, which we'll use for gathering  
# elements of the [2,2] tensor 'a' along its dim=1 
# (that is, horizontal) dimension.
i = torch.tensor([1,0])
print(i)

# note that since we're gathering along the dim=1 dimension
# of 'a' the tensor 'i' must be of the same shape as 'a'
# along other dimensions besides dim=1, that is, along dim=0.
# by using view(-1,...) we make the dim=0 (vertical)
# dimension of i be of the same length as the dim=0 
# dimension of 'a'.
b  = a.gather(dim=1, index=i.view(-1, 1))
print(b)

# now we get rid of extraneous dimensions.
# compare tensors 'b' and 'c'. in 'b' there is only one 
# element per row along dim=1, so its shape is [2, 1]. 
# We are, therefore, free to squeeze away the dim=1 to
# arrive at a new tensor 'c' with shape [2,]
c = b.squeeze()
print(c)

tensor([[1., 2.],
        [3., 4.]])
tensor([1, 0])
tensor([[2.],
        [3.]])
tensor([2., 3.])


### Return a (random) batch of data from the training set

In [44]:
def get_batch(x, t, batch_size):
    # the numpy function choice(length, number)
    # selects at random "batch_size" integers from 
    # the range [0, length-1] corresponding to the
    # row indices.
    rows    = rnd.choice(len(x), batch_size)
    batch_x = x[rows]
    batch_t = t[rows]
    return (batch_x, batch_t)

### Empirical risk (that is, average loss)

The empirical risk, which is the __objective function__ we shall minimize, is defined as

\begin{align}
R_M(\theta) & = \frac{1}{M}\sum_{m=1}^M L(t_m, f_m),
\end{align}

where 

\begin{align}
    f_m & \equiv f(\mathbf{x}_m, \theta),\\ \\ \textrm{and} \\
    L(t, f) &= (t - f)^2
\end{align}

The empirical risk $R_M$ approximates the __risk__

\begin{align}
R[f] & = \int \cdots \int \, p(t, \mathbf{x}) \, L(t, f(\mathbf{x}, \theta)) \, dt \, d\mathbf{x},
\end{align}

which is a __functional__ of the model $f$. The quantity $p(t, \mathbf{x}) \, dt\, d\mathbf{x}$ is the probability distribution from which the sample $\{ (t_m, \mathbf{x}_m), m = 1,\cdots, M \}$ is presumed to have been drawn. 

In [9]:
def average_loss(f, t):
  
    return  

In [10]:
def accuracy(f, t):

    return 

### Function to execute training loop

Note, here we use $t$ and $y$ interchangeably to denote the targets

In [47]:
def train(model, optimizer, avloss, getbatch,
          train_x, train_t, 
          valid_x, valid_t,
          batch_size, 
          n_iterations, step=10):
    
    xx   = []
    yy_t = []
    yy_v = []
    n    = 5000
    
    # set mode to training so that training specific operations such 
    # as dropout are enabled.
    model.train()
    
    print("%10s\t%10s\t%10s" % \
          ('iteration', 'train-set', 'valid-set'))
    
    for ii in range(n_iterations):

        # Get a random sample (a batch) of images (as numpy arrays)
        batch_x, batch_t = getbatch(train_x, train_t, batch_size)
        
        # Convert the numpy arrays batch_x and batch_t, to tensor 
        # types. The PyTorch tensor type is the magic that permits 
        # automatic differentiation with respect to parameters. 
        # However, since we do not need to take the derivatives
        # with respect to x and t, we disable this feature
        with torch.no_grad(): # no need to compute gradients wrt. x and y
            x = torch.from_numpy(batch_x).float()
            t = torch.from_numpy(batch_t).long()      

        # compute the output of the model for the batch of data x
        outputs = model(x)
        
        # compute a noisy approximation to the average loss
        empirical_risk = avloss(outputs, t)
        
        # use automatic differentiation to compute a 
        # noisy approximation of the local gradient
        optimizer.zero_grad()       # clear previous gradients
        empirical_risk.backward()   # compute gradients
        
        # Finally, advance one step in the direction of steepest 
        # descent, using the noisy local gradient. 
        optimizer.step()            # move one step
        
        if ii % step == 0:
            acc_t = validate(model, train_x[:n], train_t[:n]) 
            acc_v = validate(model, valid_x[:n], valid_t[:n])
            
            print("\r%10d\t%10.4f\t%10.4f" % (ii, acc_t, acc_v), end='')
        
            xx.append(ii)
            yy_t.append(acc_t)
            yy_v.append(acc_v)
            
    return (xx, yy_t, yy_v)

In [55]:
def validate(model, inputs, targets):
    # make sure we set evaluation mode so that training specific
    # operations such as dropout are disabled.
    model.eval() # evaluation mode
    
    with torch.no_grad(): # no need to compute gradients wrt. x and y
        # compute accuracy using training sample
        x = torch.from_numpy(inputs).float()
        t = torch.from_numpy(targets).long()
        o = model(x)
    return accuracy(o, t)

matplotlib has two graphics systems: 1) function-based and 2) object-based. The function below (plot_accuracy) illustrates the function-based system, while plot_empirical_risk illustrates the object-based system.

In [56]:
def plot_accuracy(traces):
    
    xx, yy_t, yy_v = traces
    
    # create an empty figure
    fig = plt.figure(figsize=(5, 5))
    # add a subplot to it
    nrows, ncols, index = 1,1,1
    ax  = fig.add_subplot(nrows,ncols,index)
    # adjust y limits
    axes = ax.axes
    axes.set_ylim((0, 1))
    axes.set_xlim((0, xx[-1]))
    
    plt.plot(xx, yy_t, 'b', label='Training')
    plt.plot(xx, yy_v, 'r', label='Validation')
    plt.title('Training and Validation Errors')
    plt.xlabel('Iterations', fontsize=16)
    plt.ylabel('Accuracy', fontsize=16)
    plt.grid(True, which="both", linestyle='-')
    plt.legend(loc='lower right')
    plt.show()

In [50]:
def plot_empirical_risk(xx, yy_t, yy_v):
    
    # create an empty figure
    fig = plt.figure(figsize=(8, 5))
    fig.tight_layout()
    
    # add a subplot to it
    nrows, ncols, index = 1,1,1
    ax  = fig.add_subplot(nrows,ncols,index)

    ax.set_title("Average loss function")
    
    ax.plot(xx, yy_t, 'b', lw=2, label='Training')
    ax.plot(xx, yy_v, 'r', lw=2, label='Validation')

    ax.set_xlabel('Iterations', fontsize=FONTSIZE)
    ax.set_ylabel('loss', fontsize=FONTSIZE)
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.grid(True, which="both", linestyle='-')
    ax.legend(loc='upper right')

    plt.show()

### Define model $f(\mathbf{x}, \theta)$

In [11]:
N_INPUTS  = 2       # number of inputs (x1, x2)
N_NODES_0 = 2       # number of nodes in layer 0     
N_OUTPUTS = 1

# Instead of create our own class, let's just use the Sequential class.
# Try to guess what's going on here.
model = torch.nn.Sequential(
             torch.nn.Linear(N_INPUTS,  N_NODES_0),
             torch.nn.ReLU(),
             torch.nn.Linear(N_NODES_0, N_OUTPUTS),
             torch.nn.Sigmoid()
     )

print (model)

Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): ReLU()
  (2): Linear(in_features=2, out_features=1, bias=True)
  (3): Sigmoid()
)


### Train!

Instantiate an optimizer, then train

In [ ]:
n_batch = 10
n_iterations  = 100
learning_rate = 3.e-3

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

traces = train(model, optimizer, 
               average_loss,
               get_batch,
               train_x, train_t, 
               valid_x, valid_t,
               n_batch, 
               n_iterations,
               step=10)

 iteration	 train-set	 valid-set
      3010	    0.5023	    0.4998